## ⚠️ IMPORTANT: How to Use This Notebook

### 🔧 First Time Setup (or after dependency issues)

1. **Execute Cell 5** (Install Dependencies) - This will install numpy 1.x, pandas 2.0.3, psycopg2-binary
2. **RESTART THE KERNEL** ← CRITICAL! (Menu → Kernel → Restart Kernel)
3. **Skip cells 1-6**, start from Cell 7 (## 1. Environment Setup)
4. Execute cells 7-31 in sequence

### 🚀 Normal Usage (after dependencies are installed)

**Option A - Clean Start:**
- Restart kernel
- Execute cells 7-31 in sequence

**Option B - Continue Running Kernel:**
- Execute cells in sequence from 7 onwards
- If you get numpy/pandas errors, execute cell 20 before cell 21

### 🐛 Troubleshooting

**Error: "numpy.dtype size changed, may indicate binary incompatibility"**
- Cause: numpy 2.x is installed (incompatible with pandas 2.0.3)
- Solution: Run Cell 5, then RESTART KERNEL, skip to Cell 7

**Error: "ModuleNotFoundError: No module named 'psycopg2._psycopg'"**
- Cause: Loading psycopg2 from wrong Python version
- Solution: Run Cell 20 (clears cache), then run Cell 21

See `TROUBLESHOOTING_NUMPY.md` for detailed explanation.

---

## Diagnostic - Check Python Version

Verify which Python version the kernel is actually using.

In [18]:
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Python version info: {sys.version_info}")
print(f"\nFirst 5 sys.path entries:")
for i, path in enumerate(sys.path[:5]):
    print(f"  {i}: {path}")

Python executable: /usr/bin/python3.11
Python version: 3.11.12 (main, Apr  9 2025, 08:55:55) [GCC 13.3.0]
Python version info: sys.version_info(major=3, minor=11, micro=12, releaselevel='final', serial=0)

First 5 sys.path entries:
  0: /home/davi/.local/lib/python3.11/site-packages
  1: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/plugins
  2: /tmp/spark-2de69f26-d690-439f-bf55-45a29dde18f2/userFiles-7ede4014-2717-423a-80fd-150467d7b67a
  3: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/helpers
  4: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/spark_config


In [19]:
# Verificar versões instaladas e compatibilidade
import sys
import subprocess

print(f"Python: {sys.executable}")
print(f"Version: {sys.version}\n")

# Check installed packages
result = subprocess.run(
    [sys.executable, "-m", "pip", "list", "--format=freeze"],
    capture_output=True,
    text=True
)

print("Installed packages:")
for line in result.stdout.split('\n'):
    if any(pkg in line.lower() for pkg in ['numpy', 'pandas', 'psycopg']):
        print(f"  {line}")

# Try importing to see actual versions
print("\nActual loaded versions:")
try:
    import numpy
    print(f"  ✅ numpy: {numpy.__version__}")
    print(f"     Location: {numpy.__file__}")
    
    # Check numpy compatibility
    numpy_major = int(numpy.__version__.split('.')[0])
    if numpy_major >= 2:
        print(f"  ⚠️  WARNING: numpy 2.x is INCOMPATIBLE with pandas 2.0.3!")
        print(f"     You must install numpy 1.x (e.g., 1.26.4)")
except Exception as e:
    print(f"  ❌ numpy: ERROR - {e}")

try:
    import pandas
    print(f"  ✅ pandas: {pandas.__version__}")
except Exception as e:
    print(f"  ❌ pandas: ERROR - {e}")

try:
    import psycopg2
    print(f"  ✅ psycopg2: {psycopg2.__version__}")
except Exception as e:
    print(f"  ❌ psycopg2: ERROR - {e}")

print("\n" + "="*60)
print("COMPATIBILITY CHECK:")
print("="*60)
try:
    import numpy
    import pandas
    numpy_ver = tuple(map(int, numpy.__version__.split('.')[:2]))
    pandas_ver = tuple(map(int, pandas.__version__.split('.')[:2]))
    
    if numpy_ver[0] >= 2 and pandas_ver == (2, 0):
        print("❌ INCOMPATIBLE: numpy 2.x + pandas 2.0.x will cause binary errors!")
        print("   Solution: Install numpy 1.x with: pip install 'numpy<2.0'")
    elif numpy_ver[0] == 1 and pandas_ver == (2, 0):
        print("✅ COMPATIBLE: numpy 1.x + pandas 2.0.x is the correct combination")
    else:
        print(f"⚠️  Unknown combination: numpy {numpy.__version__} + pandas {pandas.__version__}")
except:
    print("⚠️  Could not check compatibility")
print("="*60)


Python: /usr/bin/python3.11
Version: 3.11.12 (main, Apr  9 2025, 08:55:55) [GCC 13.3.0]

Installed packages:
  numpy==1.26.4
  pandas==2.0.3
  psycopg2-binary==2.9.9

Actual loaded versions:
  ✅ numpy: 1.26.4
     Location: /home/davi/.local/lib/python3.11/site-packages/numpy/__init__.py
  ✅ pandas: 2.0.3
  ✅ psycopg2: 2.9.9 (dt dec pq3 ext lo64)

COMPATIBILITY CHECK:
✅ COMPATIBLE: numpy 1.x + pandas 2.0.x is the correct combination
Installed packages:
  numpy==1.26.4
  pandas==2.0.3
  psycopg2-binary==2.9.9

Actual loaded versions:
  ✅ numpy: 1.26.4
     Location: /home/davi/.local/lib/python3.11/site-packages/numpy/__init__.py
  ✅ pandas: 2.0.3
  ✅ psycopg2: 2.9.9 (dt dec pq3 ext lo64)

COMPATIBILITY CHECK:
✅ COMPATIBLE: numpy 1.x + pandas 2.0.x is the correct combination


In [20]:
# Install required packages for PostgreSQL connection
import subprocess
import sys

def uninstall_package(package_name):
    """Uninstall a package completely."""
    try:
        print(f"🗑️  Uninstalling {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", package_name])
        print(f"✅ {package_name} uninstalled")
        return True
    except Exception as e:
        print(f"⚠️  {package_name} was not installed or error: {e}")
        return False

def install_package(package):
    """Install a package with force-reinstall."""
    package_name = package.split('==')[0].split('<')[0].split('>')[0]
    print(f"📦 Installing {package}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", "--force-reinstall", "--no-deps", package])
        print(f"✅ {package} installed!")
        return True
    except Exception as e:
        print(f"❌ Failed to install {package}: {e}")
        return False

print("="*60)
print("FIXING DEPENDENCIES")
print("="*60)

# First, completely remove broken packages
print("\n🔧 Step 1: Removing broken installations...")
uninstall_package("psycopg2")
uninstall_package("psycopg2-binary")
uninstall_package("pandas")
uninstall_package("numpy")

# Install fresh packages in the correct order
# CRITICAL: numpy MUST be exactly 1.26.4 (NOT 2.x!)
print("\n🔧 Step 2: Installing fresh packages...")
packages = [
    "numpy==1.26.4",  # EXACT version - 2.x is INCOMPATIBLE with pandas 2.0.3!
    "pandas==2.0.3",
    "psycopg2-binary==2.9.9"
]

all_ok = True
for pkg in packages:
    if not install_package(pkg):
        all_ok = False

# Install dependencies of pandas and numpy
if all_ok:
    print("\n🔧 Step 3: Installing dependencies...")
    deps = ["python-dateutil", "pytz", "tzdata", "six"]
    for dep in deps:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", dep], 
                                stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except:
            pass

print("\n" + "="*60)
if all_ok:
    print("✅ All dependencies installed successfully!")
    print("\n⚠️  CRITICAL: RESTART THE KERNEL NOW!")
    print("   1. Menu → Kernel → Restart Kernel")
    print("   2. After restart, skip cells 1-6")
    print("   3. Start from Cell 7 (## 1. Environment Setup)")
    print("   4. Execute cells 7-31 in sequence")
    print("\n💡 TIP: If numpy 2.x gets reinstalled again, run:")
    print("   bash silver/install_dependencies.sh")
else:
    print("❌ Some packages failed to install")
print("="*60)


FIXING DEPENDENCIES

🔧 Step 1: Removing broken installations...
🗑️  Uninstalling psycopg2...


✅ psycopg2 uninstalled
🗑️  Uninstalling psycopg2-binary...
Found existing installation: psycopg2-binary 2.9.9
Uninstalling psycopg2-binary-2.9.9:
  Successfully uninstalled psycopg2-binary-2.9.9
✅ psycopg2-binary uninstalled
🗑️  Uninstalling pandas...
Found existing installation: psycopg2-binary 2.9.9
Uninstalling psycopg2-binary-2.9.9:
  Successfully uninstalled psycopg2-binary-2.9.9
✅ psycopg2-binary uninstalled
🗑️  Uninstalling pandas...
Found existing installation: pandas 2.0.3
Uninstalling pandas-2.0.3:
  Successfully uninstalled pandas-2.0.3
✅ pandas uninstalled
🗑️  Uninstalling numpy...
Found existing installation: pandas 2.0.3
Uninstalling pandas-2.0.3:
  Successfully uninstalled pandas-2.0.3
✅ pandas uninstalled
🗑️  Uninstalling numpy...
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
✅ numpy uninstalled

🔧 Step 2: Installing fresh packages...
📦 Installing numpy==1.26.4...
Found existing installation: numpy 1.26.4
Un

# Test Postgres Insert - Silver Layer

Simple test notebook to validate PostgreSQL integration with PySpark.

This notebook demonstrates:
1. Creating a test DataFrame with PySpark
2. Connecting to PostgreSQL using postgres_helper
3. Inserting data using cliente_postgres

## 1. Environment Setup

**CRITICAL**: This cell ensures we use packages from the SAME Python version as the kernel.
If the kernel uses Python 3.11, we MUST use packages from Python 3.11 site-packages.
Using packages from a different Python version (e.g., 3.12) causes binary incompatibility errors.

In [21]:
# Configure environment for local execution
import os
import sys
from pathlib import Path

if 'AIRFLOW_HOME' not in os.environ:
    # Running locally - determine the CORRECT Python version being used by kernel
    kernel_python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
    print(f"🐍 Kernel using Python {kernel_python_version}")
    print(f"📍 Executable: {sys.executable}")
    
    # Define user site-packages for the kernel's Python version
    user_site_packages = Path.home() / '.local' / 'lib' / f'python{kernel_python_version}' / 'site-packages'
    
    # CRITICAL: Remove /usr/lib/python3/dist-packages (contains broken system numpy)
    # and any paths from other Python versions
    paths_to_remove = []
    for path_str in sys.path:
        # Remove other Python versions
        if 'python3.' in path_str and kernel_python_version not in path_str:
            paths_to_remove.append(path_str)
        # Remove system dist-packages (contains broken numpy on Ubuntu 24.04)
        elif path_str == '/usr/lib/python3/dist-packages':
            paths_to_remove.append(path_str)
    
    for path in paths_to_remove:
        try:
            sys.path.remove(path)
            print(f"🗑️  Removed incompatible path: {path}")
        except ValueError:
            pass
    
    # Ensure user site-packages is first (after current directory)
    if user_site_packages.exists():
        if str(user_site_packages) in sys.path:
            sys.path.remove(str(user_site_packages))
        sys.path.insert(0, str(user_site_packages))
        print(f"✅ Prioritized user packages: {user_site_packages}")
    
    # Try to find PySpark in multiple locations
    pyspark_found = False
    search_paths = [
        user_site_packages,
        Path.home() / '.local' / 'lib' / 'python3.12' / 'site-packages',
        Path('/usr/local/lib') / f'python{kernel_python_version}' / 'dist-packages',
        Path('/usr/local/lib/python3.12/dist-packages'),
    ]
    
    for search_path in search_paths:
        pyspark_path = search_path / 'pyspark'
        if pyspark_path.exists():
            if str(search_path) not in sys.path:
                sys.path.insert(0, str(search_path))
            print(f"✅ PySpark found at: {pyspark_path}")
            pyspark_found = True
            break
    
    if not pyspark_found:
        print("❌ PySpark not found in any location!")
        print("   Please install with: pip install --user pyspark==3.5.1")
    
    # Verify import
    try:
        import pyspark
        print(f"✅ PySpark {pyspark.__version__} loaded successfully")
    except ImportError as e:
        print(f"❌ Error importing PySpark: {e}")
    
    # Configure Java
    java_home = '/usr/lib/jvm/java-17-openjdk-amd64'
    if os.path.exists(java_home):
        os.environ['JAVA_HOME'] = java_home
        print(f"✅ Java configured: {java_home}")
    else:
        print("⚠️  Warning: Java 17 not found at expected location")
        print("   Install with: sudo apt install openjdk-17-jdk")
    
    print("\n📋 Final sys.path (first 5):")
    for i, p in enumerate(sys.path[:5]):
        print(f"   {i}: {p}")
else:
    print("🐳 Running in Airflow - environment already configured")


🐍 Kernel using Python 3.11
📍 Executable: /usr/bin/python3.11
✅ Prioritized user packages: /home/davi/.local/lib/python3.11/site-packages
✅ PySpark found at: /home/davi/.local/lib/python3.12/site-packages/pyspark
✅ PySpark 3.5.1 loaded successfully
✅ Java configured: /usr/lib/jvm/java-17-openjdk-amd64

📋 Final sys.path (first 5):
   0: /home/davi/.local/lib/python3.12/site-packages
   1: /home/davi/.local/lib/python3.11/site-packages
   2: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/plugins
   3: /tmp/spark-2de69f26-d690-439f-bf55-45a29dde18f2/userFiles-7ede4014-2717-423a-80fd-150467d7b67a
   4: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/helpers


In [22]:
# CRITICAL CHECK: Verify numpy version BEFORE proceeding
import sys

print("🔍 Checking numpy version...")
try:
    import numpy
    numpy_version = numpy.__version__
    numpy_major = int(numpy_version.split('.')[0])
    
    print(f"   Installed: numpy {numpy_version}")
    print(f"   Location: {numpy.__file__}")
    
    if numpy_major >= 2:
        print("\n" + "="*60)
        print("❌ CRITICAL ERROR: INCOMPATIBLE NUMPY VERSION!")
        print("="*60)
        print(f"Detected: numpy {numpy_version}")
        print("Required: numpy 1.26.4")
        print("\nPandas 2.0.3 is INCOMPATIBLE with numpy 2.x!")
        print("This will cause: ValueError: numpy.dtype size changed")
        print("\n🔧 FIX:")
        print("   Option 1: Run Cell 5 (Install Dependencies), then RESTART KERNEL")
        print("   Option 2: Run in terminal:")
        print("      python3.11 -m pip uninstall -y numpy")
        print("      python3.11 -m pip install --user 'numpy==1.26.4'")
        print("   Option 3: Run script:")
        print("      bash silver/install_dependencies.sh")
        print("\nAfter fixing, RESTART THE KERNEL before continuing!")
        print("="*60)
        raise RuntimeError(f"numpy {numpy_version} is incompatible with pandas 2.0.3")
    else:
        print(f"✅ numpy {numpy_version} is compatible with pandas 2.0.3")
        
except ImportError:
    print("⚠️  numpy not installed!")
    print("   Run Cell 5 (Install Dependencies), then RESTART KERNEL")
    raise


🔍 Checking numpy version...
   Installed: numpy 1.26.4
   Location: /home/davi/.local/lib/python3.11/site-packages/numpy/__init__.py
✅ numpy 1.26.4 is compatible with pandas 2.0.3


In [23]:
import sys
import os
from pathlib import Path

# Define paths
if 'AIRFLOW_HOME' in os.environ:
    # Running in Airflow
    BASE_PATH = Path('/opt/airflow')
    SPARK_CONFIG_PATH = BASE_PATH / 'spark_config'
    POSTGRES_HELPERS_PATH = BASE_PATH / 'postgres' / 'helpers'
    POSTGRES_PLUGINS_PATH = BASE_PATH / 'postgres' / 'plugins'
else:
    # Running manually
    BASE_PATH = Path.cwd().parent
    SPARK_CONFIG_PATH = BASE_PATH / 'spark_config'
    POSTGRES_HELPERS_PATH = BASE_PATH / 'postgres' / 'helpers'
    POSTGRES_PLUGINS_PATH = BASE_PATH / 'postgres' / 'plugins'

# Add paths
sys.path.insert(0, str(SPARK_CONFIG_PATH))
sys.path.insert(0, str(POSTGRES_HELPERS_PATH))
sys.path.insert(0, str(POSTGRES_PLUGINS_PATH))

print(f"Base Path: {BASE_PATH}")
print(f"Spark Config Path: {SPARK_CONFIG_PATH}")
print(f"Postgres Helpers Path: {POSTGRES_HELPERS_PATH}")
print(f"Postgres Plugins Path: {POSTGRES_PLUGINS_PATH}")

Base Path: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2
Spark Config Path: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/spark_config
Postgres Helpers Path: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/helpers
Postgres Plugins Path: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/plugins


In [24]:
# Configure PostgreSQL connection for manual execution
if 'AIRFLOW_HOME' not in os.environ:
    # Set PostgreSQL connection environment variables
    os.environ.setdefault('POSTGRES_DB', 'data_warehouse')
    os.environ.setdefault('POSTGRES_USER', 'airflow')
    os.environ.setdefault('POSTGRES_PASSWORD', 'airflow')
    os.environ.setdefault('POSTGRES_HOST', 'localhost')
    os.environ.setdefault('POSTGRES_PORT', '5433')
    
    print("✅ PostgreSQL environment variables configured")
    print(f"   Database: {os.environ['POSTGRES_DB']}")
    print(f"   Host: {os.environ['POSTGRES_HOST']}:{os.environ['POSTGRES_PORT']}")
else:
    print("🐳 Running in Airflow - using Airflow connection settings")

✅ PostgreSQL environment variables configured
   Database: data_warehouse
   Host: localhost:5433


## 2. Initialize Spark Session

In [25]:
from config import SparkConfig
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Create Spark session
spark_config = SparkConfig(app_name="Test_Postgres_Insert")
spark = spark_config.create_spark_session()
spark_config.configure_for_banking_data()

print(f"Spark Version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

INFO:config:Spark Session criada: Test_Postgres_Insert
INFO:config:Spark UI disponível em: http://192.168.0.12:4041
INFO:config:Spark UI disponível em: http://192.168.0.12:4041


Spark Version: 3.5.1
Spark UI: http://192.168.0.12:4041


## 3. Create Test DataFrame

In [26]:
# Create a simple test DataFrame
test_data = [
    ("Product A", 100, 25.50),
    ("Product B", 200, 15.75),
    ("Product C", 150, 30.00),
    ("Product D", 300, 10.25),
    ("Product E", 50, 45.99)
]

# Define schema
schema = StructType([
    StructField("product_name", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", DoubleType(), True)
])

# Create DataFrame
df_test = spark.createDataFrame(test_data, schema)

print("✅ Test DataFrame created successfully!")
print(f"Total records: {len(test_data)}")

✅ Test DataFrame created successfully!
Total records: 5


## 4. Display DataFrame Schema

In [27]:
# Display schema
df_test.printSchema()

root
 |-- product_name: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: double (nullable = true)



## 5. Show Sample Data

In [28]:
# Display sample records
print("Sample records:")
df_test.show(truncate=False)

Sample records:
+------------+--------+-----+
|product_name|quantity|price|
+------------+--------+-----+
|Product A   |100     |25.5 |
|Product B   |200     |15.75|
|Product C   |150     |30.0 |
|Product D   |300     |10.25|
|Product E   |50      |45.99|
+------------+--------+-----+

+------------+--------+-----+
|product_name|quantity|price|
+------------+--------+-----+
|Product A   |100     |25.5 |
|Product B   |200     |15.75|
|Product C   |150     |30.0 |
|Product D   |300     |10.25|
|Product E   |50      |45.99|
+------------+--------+-----+



## 6. Connect to PostgreSQL

In [29]:
# CRITICAL: Force remove Python 3.12 paths before importing psycopg2
# ⚠️  WARNING: ALWAYS RUN THIS CELL BEFORE IMPORTING POSTGRESQL LIBRARIES!
# This cell MUST be executed before the next cell to prevent numpy/pandas version conflicts.
import sys
from pathlib import Path

kernel_version = f"{sys.version_info.major}.{sys.version_info.minor}"
print(f"🔧 Preparing environment for Python {kernel_version}")

# Define the correct user site-packages path
user_site = str(Path.home() / '.local' / 'lib' / f'python{kernel_version}' / 'site-packages')

# CRITICAL: Remove ALL problematic paths
paths_to_remove = []
for path_str in sys.path:
    # Remove other Python versions (3.12, 3.13, etc.)
    if 'python3.' in path_str and kernel_version not in path_str:
        paths_to_remove.append(path_str)
    # Remove /usr/lib/python3/dist-packages (BROKEN numpy on Ubuntu 24.04)
    elif path_str == '/usr/lib/python3/dist-packages':
        paths_to_remove.append(path_str)
    # Remove /usr/local/lib/python3/dist-packages if it exists
    elif path_str == '/usr/local/lib/python3/dist-packages':
        paths_to_remove.append(path_str)

for path in paths_to_remove:
    try:
        sys.path.remove(path)
        print(f"🗑️  Removed: {path}")
    except ValueError:
        pass

# Ensure Python 3.11 user site-packages is FIRST
if user_site in sys.path:
    sys.path.remove(user_site)
sys.path.insert(0, user_site)
print(f"✅ Prioritized: {user_site}")

# Clear ALL cached imports that might have incompatible binaries
modules_to_clear = [
    'psycopg2', 'psycopg2._psycopg', 'psycopg2.extras',
    'pandas', 'pandas.core', 'pandas._libs',
    'numpy', 'numpy.core', 'numpy.core._multiarray_umath',
    'cliente_postgres', 'postgres_helper'
]

cleared_count = 0
for module_name in list(sys.modules.keys()):
    # Clear exact matches or submodules
    if module_name in modules_to_clear or any(module_name.startswith(m + '.') for m in modules_to_clear):
        del sys.modules[module_name]
        cleared_count += 1

print(f"🔄 Cleared {cleared_count} cached modules")

print("\n📋 sys.path (first 5):")
for i, p in enumerate(sys.path[:5]):
    print(f"  {i}: {p}")

print("\n✅ Environment ready for PostgreSQL imports")


🔧 Preparing environment for Python 3.11
🗑️  Removed: /home/davi/.local/lib/python3.12/site-packages
✅ Prioritized: /home/davi/.local/lib/python3.11/site-packages
🔄 Cleared 407 cached modules

📋 sys.path (first 5):
  0: /home/davi/.local/lib/python3.11/site-packages
  1: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/plugins
  2: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/helpers
  3: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/spark_config
  4: /home/davi/Área de Trabalho/bancos2/TrabalhoSBD2/postgres/plugins

✅ Environment ready for PostgreSQL imports


In [30]:
# Import postgres helper and client
from postgres_helper import get_postgres_conn
from cliente_postgres import ClientPostgresDB

# Get connection string
conn_str = get_postgres_conn()
print("✅ PostgreSQL connection string obtained")

# Create client
postgres_client = ClientPostgresDB(conn_str)
print("✅ PostgreSQL client created")

/home/davi/.local/lib/python3.11/site-packages/pandas/__init__.py:11: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  __import__(_dependency)
INFO:root:[postgres_helpers] Using manual PostgreSQL connection: dbname=data_warehouse, user=airflow, host=localhost, port=5433


✅ PostgreSQL connection string obtained


INFO:root:[cliente_postgres.py] Initialized ClientPostgresDB with conn_str: dbname=data_warehouse user=airflow password=airflow host=localhost port=5433


✅ PostgreSQL client created


## 7. Convert DataFrame to Dictionary List

In [31]:
# Convert Spark DataFrame to list of dictionaries
rows = df_test.collect()
data_to_insert = [row.asDict() for row in rows]

print(f"✅ Data converted successfully!")
print(f"Number of records: {len(data_to_insert)}")
print(f"Sample record: {data_to_insert[0]}")

✅ Data converted successfully!
Number of records: 5
Sample record: {'product_name': 'Product A', 'quantity': 100, 'price': 25.5}


## 8. Insert Data into PostgreSQL

In [32]:
# Define table name and schema
table_name = "test_products"
schema_name = "silver"

# Insert data
postgres_client.insert_data(
    data=data_to_insert,
    table_name=table_name,
    schema=schema_name,
    primary_key=["product_name"],
    conflict_fields=["product_name"]
)

print(f"✅ Data inserted into {schema_name}.{table_name}")

INFO:root:[cliente_postgres.py] Schema silver ensured to exist
INFO:root:[cliente_postgres.py] Table silver.test_products created or already exists
INFO:root:[cliente_postgres.py] Table silver.test_products created or already exists
INFO:root:[cliente_postgres.py] Inserted data into silver.test_products
INFO:root:[cliente_postgres.py] Inserted data into silver.test_products


✅ Data inserted into silver.test_products


## 9. Verify Data Insertion

In [33]:
# Query the data
query = f"SELECT * FROM {schema_name}.{table_name} ORDER BY product_name;"
results = postgres_client.execute_query(query)

print(f"✅ Data retrieved from {schema_name}.{table_name}")
print(f"Total records in database: {len(results)}")
print("\nData in database:")
for row in results:
    print(row)

INFO:root:[cliente_postgres.py] Executing query: SELECT * FROM silver.test_products ORDER BY product_name;
INFO:root:[cliente_postgres.py] Query executed successfully, fetched 5 rows
INFO:root:[cliente_postgres.py] Query executed successfully, fetched 5 rows


✅ Data retrieved from silver.test_products
Total records in database: 5

Data in database:
('Product A', '100', '25.5')
('Product B', '200', '15.75')
('Product C', '150', '30.0')
('Product D', '300', '10.25')
('Product E', '50', '45.99')


## 10. Summary Report

In [34]:
# Generate summary report
print("="*60)
print("TEST SUMMARY")
print("="*60)
print(f"DataFrame records created: {len(test_data)}")
print(f"Records inserted to database: {len(data_to_insert)}")
print(f"Records retrieved from database: {len(results)}")
print(f"Table: {schema_name}.{table_name}")
print("="*60)
print("✅ All operations completed successfully!")

TEST SUMMARY
DataFrame records created: 5
Records inserted to database: 5
Records retrieved from database: 5
Table: silver.test_products
✅ All operations completed successfully!


## 11. Cleanup

In [35]:
# Stop Spark session
spark_config.stop_session()
print("Test completed successfully!")

INFO:config:Spark Session finalizada


Test completed successfully!
